In [9]:
import cv2
import numpy as np
from ultralytics import YOLO
from vidgear.gears import CamGear
import cvzone
from polym import PolylineManager       # Ensure this imports the class correctly

In [ ]:

# Initialize the video stream
stream = CamGear(source='https://www.youtube.com/watch?v=_TusTf0iZQU', stream_mode=True, logging=True).start()

# Load COCO class names
with open("coco.txt", "r") as f:
    class_names = f.read().splitlines()

# Load the YOLOv8 model
model = YOLO("yolo11s.pt")

# Create a PolylineManager instance
polyline_manager = PolylineManager()

# Set up the OpenCV window
cv2.namedWindow('RGB')

# Mouse callback to get mouse movements
def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:  # Add point on left mouse click
        polyline_manager.add_point((x, y))

# Set the mouse callback function
cv2.setMouseCallback('RGB', RGB)

going_up = {}
going_down = {}
gnu=[]
gnd=[]
while True:
    # Read a frame from the video stream
    frame = stream.read()
    

    frame = cv2.resize(frame, (1020, 500))

    # Run YOLOv8 tracking on the frame
    results = model.track(frame, persist=True, classes=[2])

    # Check if there are any boxes in the results
    if results[0].boxes is not None and results[0].boxes.id is not None:
        # Get the boxes (x, y, w, h), class IDs, track IDs, and confidences
        boxes = results[0].boxes.xyxy.int().cpu().tolist()  # Bounding boxes
        class_ids = results[0].boxes.cls.int().cpu().tolist()  # Class IDs
        track_ids = results[0].boxes.id.int().cpu().tolist()  # Track IDs
        confidences = results[0].boxes.conf.cpu().tolist()  # Confidence score

        # Draw boxes and labels on the frame
        for box, class_id, track_id, conf in zip(boxes, class_ids, track_ids, confidences):
            c = class_names[class_id]
            x1, y1, x2, y2 = box
            
            # Calculate the center of the bounding box
            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2
            
            
            if polyline_manager.point_polygon_test((cx, cy), 'area1'):
                going_up[track_id] = (cx, cy)
            if track_id in going_up:
               if polyline_manager.point_polygon_test((cx, cy), 'area2'): 
                  cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                  cvzone.putTextRect(frame, f'{track_id}', (x1, y2), 1, 1)
                  cvzone.putTextRect(frame, f'{c}', (x1, y1), 1, 1)
                  if gnu.count(track_id)==0:
                     gnu.append(track_id)
                     
            if polyline_manager.point_polygon_test((cx, cy), 'area2'):
                going_down[track_id] = (cx, cy)
            if track_id in going_down:
               if polyline_manager.point_polygon_test((cx, cy), 'area1'): 
                  cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 255), 2)
                  cvzone.putTextRect(frame, f'{track_id}', (x1, y2), 1, 1)
                  cvzone.putTextRect(frame, f'{c}', (x1, y1), 1, 1)
                  if gnd.count(track_id)==0:
                     gnd.append(track_id)
                
    godown=len(gnd)       
    goup=len(gnu)
    cvzone.putTextRect(frame, f'GoDown: {godown}', (50, 60), 2, 2)
    cvzone.putTextRect(frame, f'GoUp: {goup}', (50, 160), 2, 2)

    # Draw polylines and points on the frame
    frame = polyline_manager.draw_polylines(frame)

    # Display the frame
    cv2.imshow("RGB", frame)

    # Handle key events for polyline management
    if not polyline_manager.handle_key_events():
        break

# Release the video capture object and close the display window
stream.stop()
cv2.destroyAllWindows()

23:55:56 ::    Helper     ::   INFO   :: Running VidGear Version: 0.3.3
23:55:56 ::    Helper     ::  DEBUG   :: Selecting `best` resolution for streams.
23:55:56 ::    CamGear    ::   INFO   :: Verifying Streaming URL using yt-dlp backend. Please wait...
23:55:59 ::    CamGear    ::   INFO   :: [Backend] :: Streaming URL is fully supported. Available Streams are: [144p, 240p, 360p, 480p, 720p, 1080p, best, worst]
23:55:59 ::    CamGear    :: WARNING  :: Livestream URL detected. It is strongly recommended to use the GStreamer backend (`backend=cv2.CAP_GSTREAMER`) with these URLs.
23:55:59 ::    CamGear    ::  DEBUG   :: Using `best` resolution for streaming.
23:55:59 ::    CamGear    ::  DEBUG   :: YouTube source ID: `_TusTf0iZQU`, Title: `г. Усть-Кут Online, Иркутская область - Пешеходный переход Школа #10 2024-11-04 23:55`, Quality: `best`
23:55:59 ::    CamGear    ::  DEBUG   :: Enabling Threaded Queue Mode for the current video source!



0: 320x640 (no detections), 308.5ms
Speed: 3.0ms preprocess, 308.5ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 162.4ms
Speed: 2.5ms preprocess, 162.4ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 166.1ms
Speed: 2.1ms preprocess, 166.1ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 189.6ms
Speed: 2.0ms preprocess, 189.6ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 190.8ms
Speed: 2.1ms preprocess, 190.8ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 176.6ms
Speed: 1.1ms preprocess, 176.6ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 183.4ms
Speed: 2.0ms preprocess, 183.4ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 167.0ms
Speed: 2.0ms prepr

Enter a name for the polyline:  area1



0: 320x640 (no detections), 219.8ms
Speed: 3.1ms preprocess, 219.8ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 170.8ms
Speed: 2.5ms preprocess, 170.8ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 184.8ms
Speed: 2.1ms preprocess, 184.8ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 196.8ms
Speed: 1.5ms preprocess, 196.8ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 316.8ms
Speed: 2.1ms preprocess, 316.8ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 254.4ms
Speed: 2.6ms preprocess, 254.4ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 319.1ms
Speed: 2.1ms preprocess, 319.1ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 325.5ms
Speed: 2.2ms prepr

Enter a name for the polyline:  area2



0: 320x640 (no detections), 62.9ms
Speed: 1.0ms preprocess, 62.9ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 80.1ms
Speed: 1.1ms preprocess, 80.1ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 91.9ms
Speed: 1.3ms preprocess, 91.9ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 128.8ms
Speed: 1.0ms preprocess, 128.8ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 189.8ms
Speed: 3.1ms preprocess, 189.8ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 180.9ms
Speed: 2.1ms preprocess, 180.9ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 213.1ms
Speed: 2.1ms preprocess, 213.1ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 197.7ms
Speed: 2.0ms preprocess,

In [1]:
pip install import-ipynb

  Obtaining dependency information for import-ipynb from https://files.pythonhosted.org/packages/ec/62/e0b830773060d2a390aa923dcc8afc680d798bdbdadb6394f760fac62517/import_ipynb-0.2-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.
